
- 📚 데이터 사이언티스트의 실전노트, 이지영, [비제이퍼블릭](https://bjpublic.tistory.com/) 
- 🌟 [YouTube: 통계학 & 데이터과학](https://https://www.youtube.com/channel/UC2BreMMPUd0djRpA4UEaD3Q)
- 💻 [클래스101: 데이터 과학자 실무 프로젝트 (분석+예측모델 + AWS 자동화)](https://class101.page.link/MhG4)


## 3.2 기본 통계로 질문자 되기
### 3.2.13 중심극한정리에서 시작하는 추리통계 (Page 254)

In [1]:
from IPython.core.display import display, HTML
def df_display(dfs:list, captions:list):
    """ 데이터 프레임을 나란히 보여줌
        dfs: 데이터 프레임 리스트
        captions: 각 데이터 테이블의 설명
    """
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0"
    display(HTML(output))

In [2]:
import numpy as np
import pandas as pd
pd.set_option("display.precision", 2) # 소수점 두자리 표현

# 경고 표시 지움 
import warnings
warnings.filterwarnings('ignore')

# 앞서 사용한 get_info 사용
def get_info(df, col_list):
    # 통계치를 저장할 빈 리스트 설정 
    list_std = []
    list_1Q = []
    list_mean = []
    list_median = []

    for col in col_list: 
        list_1Q.append(round(np.percentile(df[col], 25),2))
        list_mean.append(round(df[col].mean(),2)) # 평균
        list_median.append(round(df[col].median(),2)) # 중앙값
        list_std.append(round(df[col].std(),2)) # 표준편차
       
    # 리스트를 데이터프레임으로 변환
    df_info = pd.DataFrame({'열이름': col_list, 
                            '크기' : len(df),
                            '표준편차': list_std,
                            '1Q' : list_1Q,
                            '평균': list_mean,
                            '중앙값' : list_median}).set_index('열이름').T
    return df_info

# 랜덤시드 설정
np.random.seed(100) 

# 모집단 만들기 
N = 100000 # 모집단크기
population_x = np.random.exponential(scale=10, size= N) # 비대칭분포
df_population = pd.DataFrame({"모집단": population_x}) # 열 이름 바꾸기


In [3]:
# 표본분포 만들기 
sample_mean = []

k=70 # 70번 반복해서 모집단에서 표본 표집
n=40 # 표본크기 
for i in range(k): 
    
    # 모집단에서 무작위로 표본 추출 
    chosen_idx = np.random.choice(N, replace=False, size=n)
    x = df_population.iloc[chosen_idx] 
    
    # 표본에서 평균 계산
    avg = x["모집단"].mean()
    
    # 계산된 평균을 리스트에 추가 
    sample_mean.append(avg)
    
# 70개 각 표본 평균을로 이뤄진 표집분포 
df_sam_dist_clt = pd.DataFrame({"표집분포": sample_mean}) # 열 이름 바꾸기
    
# 공식에 따른 표준오차
print("표준오차:", round(df_population["모집단"].std()/np.sqrt(n),3))

# 모집단, 표집분포 주요 정보 비교 
df_display([get_info(df_population,["모집단"]), 
                 get_info(df_sam_dist_clt, ["표집분포"])],
                 ["모집단", "표집분포_CLT"])



표준오차: 1.58


열이름,모집단
크기,100000.00
표준편차,9.99
1Q,2.88
평균,10.00
중앙값,6.95
열이름,표집분포
크기,70.00
표준편차,1.33
1Q,9.12
평균,9.98


In [4]:
# 표본 만들기 
n = 50 # 표본 크기
chosen_idx = np.random.choice(N, replace=False, size=n) # 모집단인덱스에서 무작위 n개 선택
df_sample = df_population.iloc[chosen_idx] # 추출된 인덱스만 선택
df_sample.rename(columns={"모집단":"표본"}, inplace=True) 
df_sample.reset_index(drop=True, inplace=True) # 인덱스 값 재설정

# 부트스트래핑 표본 만들기
# 초기 리스트 생성, 차후 한번 부트스트래핑 통계치 계산 할 때마다 값 추가 예정
sample_std = []
sample_1Q = []
sample_mean = []
sample_median = []

k=70 # 70번 반복해서 표본에서 부트스트랩 표집
n_bs = 30 # 부트스트래핑 표본 크기 

# 부트스트랩 표본 만들기 
for i in range(k): 
    
    # 표본에서 무작위로 부트스트랩링 표본 추출 
    chosen_idx = np.random.choice(n, replace=True, size=n_bs)
    x = df_sample.iloc[chosen_idx] 

    # 부트스트래핑 표본에서 통계치 계산
    std = np.std(x)
    Q1 = np.percentile(x, 25)
    avg = np.mean(x)
    median = np.median(x)
    
    # 계산된 통계치를 리스트에 추가 
    sample_std.append(std)
    sample_1Q.append(Q1)
    sample_mean.append(avg)
    sample_median.append(median)
    
# 4개의 통계치가 각 부트스트래핑 표집분포를 의미
columns=['크기','표준편차','1Q', '평균','중앙값']
sam_dist = np.array([len(sample_mean), 
              np.mean(sample_std), 
              np.mean(sample_1Q), 
              np.mean(sample_mean), 
              np.mean(sample_median)])

# 통계치 정보를 차후 표로 비교하기 위해 데이터 프레임으로 변환
df_sam_dist = pd.DataFrame(sam_dist.reshape(-1, len(sam_dist)),columns=columns).T
df_sam_dist.rename(columns={0:"표집분포_BS"}, inplace=True)

df_display([get_info(df_population,["모집단"]), 
            get_info(df_sample, ["표본"]),
            df_sam_dist],
           ["모집단", "표본 1개", "표집분포_BS"])



열이름,모집단
크기,100000.00
표준편차,9.99
1Q,2.88
평균,10.00
중앙값,6.95
열이름,표본
크기,50.00
표준편차,11.40
1Q,2.83
평균,11.01


⚠ 저작권: Copyright 2022. (이지영) all rights reserved. 본 자료는 저작권법에 의하여 보호받는 저작물로서 이에 대한 무단 복제 및 배포를 원칙적으로 금합니다. 협의 없이 배포하거나 무단으로 사용할 경우 저작권법 제136조, 137조, 138조 위반으로 사전 경고 없이 손해배상 청구 등 민,형사상의 책임과 처벌을 받을 수 있습니다.